In [ ]:
import pandas as pd
import numpy as np
from random import sample
import matplotlib.pyplot as plt
from IPython.core.pylabtools import figsize
import glob
import cv2

from google.colab import drive

import matplotlib.pyplot as plt
import keras as ks
from keras import layers
from keras.models import Sequential
import tensorflow as tf
from keras.backend import categorical_crossentropy

from sklearn.metrics import accuracy_score

import os

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
image_size = (250, 250)
batch_size = 32

train_directory = '/content/drive/MyDrive/BT5153 Applied ML/Project/BT5153 Applied ML Project /5 classes 5k each for train/Training'
val_directory = '/content/drive/MyDrive/BT5153 Applied ML/Project/BT5153 Applied ML Project /5 classes 5k each for train/Validation'
test_directory = '/content/drive/MyDrive/BT5153 Applied ML/Project/BT5153 Applied ML Project /5 classes 5k each for train/Testing'

train_ds = tf.keras.preprocessing.image_dataset_from_directory(train_directory, label_mode='categorical', image_size=image_size, batch_size=batch_size)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(val_directory, label_mode='categorical', image_size=image_size, batch_size=batch_size)
test_ds = tf.keras.preprocessing.image_dataset_from_directory(test_directory, label_mode='categorical', image_size=image_size, batch_size=batch_size)

train_ds = train_ds.prefetch(buffer_size=batch_size)
val_ds = val_ds.prefetch(buffer_size=batch_size)
test_ds = test_ds.prefetch(buffer_size=batch_size)

Found 25000 files belonging to 5 classes.
Found 27447 files belonging to 5 classes.
Found 34281 files belonging to 5 classes.


In [ ]:
def make_model(input_shape, num_classes):
    inputs = ks.Input(shape=input_shape)
    x = layers.Rescaling(1.0/255)(inputs)
    x = layers.Conv2D(32, 3, strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    x = layers.Conv2D(64, 3, strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    previous_block_activation = x
    
    for size in (128, 256, 512, 728):
        x = layers.Activation('relu')(x)
        x = layers.SeparableConv2D(size, 3, padding='same')(x)
        x = layers.BatchNormalization()(x)
        
        x = layers.Activation('relu')(x)
        x = layers.SeparableConv2D(size, 3, padding='same')(x)
        x = layers.BatchNormalization()(x)      
        
        x = layers.MaxPooling2D(3, strides=2, padding='same')(x)
        
        residual = layers.Conv2D(size, 1, strides=2, padding='same')(previous_block_activation)
        x = layers.add([x, residual])
        previous_block_activation = x

    x = layers.Activation('relu')(x)
    x = layers.SeparableConv2D(1024, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.GlobalAveragePooling2D()(x)
    
    x = layers.Dropout(0.5)(x)

    outputs = layers.Dense(num_classes, activation='softmax')(x)
        
    return ks.Model(inputs, outputs)

In [ ]:
model = make_model(image_size+(3,), 5)

epochs = 1000
saved_model = '/content/drive/MyDrive/BT5153 Applied ML/Project/BT5153 Applied ML Project /5 classes 5k each for train/CNN_5C5K_{epoch}.h5'
callbacks = [ks.callbacks.ModelCheckpoint(saved_model, save_best_only=True), 
             ks.callbacks.EarlyStopping(patience=20, restore_best_weights=True)]

model.compile(optimizer='adam', loss='CategoricalCrossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(train_ds, epochs=epochs, callbacks=callbacks, validation_data=val_ds)

Epoch 1/1000
782/782 [==============================] - 9542s 12s/step - loss: 1.0432 - accuracy: 0.5903 - val_loss: 2.0412 - val_accuracy: 0.3901
Epoch 2/1000
782/782 [==============================] - 136s 173ms/step - loss: 0.7220 - accuracy: 0.7346 - val_loss: 2.6482 - val_accuracy: 0.3283
Epoch 3/1000
782/782 [==============================] - 137s 174ms/step - loss: 0.5963 - accuracy: 0.7858 - val_loss: 1.9934 - val_accuracy: 0.4799
Epoch 4/1000
782/782 [==============================] - 136s 174ms/step - loss: 0.5094 - accuracy: 0.8248 - val_loss: 1.3848 - val_accuracy: 0.6213
Epoch 5/1000
782/782 [==============================] - 138s 176ms/step - loss: 0.4396 - accuracy: 0.8477 - val_loss: 1.5580 - val_accuracy: 0.5488
Epoch 6/1000
782/782 [==============================] - 135s 172ms/step - loss: 0.3735 - accuracy: 0.8708 - val_loss: 1.3362 - val_accuracy: 0.6521
Epoch 7/1000
782/782 [==============================] - 135s 171ms/step - loss: 0.3231 - accuracy: 0.8886 - val_l

In [ ]:
true = []
pred = []
batches = int(np.ceil(34281/32))
i = 1
for image, label in test_ds:
  pred = np.concatenate([pred, 
                         np.argmax(model.predict(image), axis=-1)])
  true = np.concatenate([true,
                         np.argmax(label.numpy(), axis=-1)])
  print('{}/{}'.format(i, batches))
  i += 1

In [ ]:
accuracy_score(true, pred)

0.7739855896852484

In [ ]:
model_results = pd.DataFrame({'true':true,
                              'pred':pred})

model_results.to_csv('/content/drive/MyDrive/BT5153 Applied ML/Project/BT5153 Applied ML Project /5 classes 5k each for train/CNN_test_preds.csv')